In [1]:
%matplotlib qt

In [2]:
import matplotlib as mpl
import happi
import numpy as np
import matplotlib.pyplot as plt

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

from matplotlib import font_manager

font_dirs = ['/Users/yao/Documents/Calibri and Cambria Fonts/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# set font
plt.rcParams['font.family'] = 'Calibri'

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
plt.rc('legend', fontsize=10)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_66080/2005029716.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [3]:
wkdir = [
    # '/Users/yao/Desktop/Data/apollon23_shock_t02/',
    # '/Users/yao/Desktop/Data/apollon23_shock_t03/',
    # '/Users/yao/Desktop/Data/apollon23_shock_t04/',
    # '/Users/yao/Desktop/Data/apo23_ne5e19_v8200_Bz60_angle90_dx01_2/',
    # '/Users/yao/Desktop/Data/apo23_2D_Bz0/',
    # '/Users/yao/Desktop/Data/apo23_ne5e19_v8200_Bz60_angle90_dx005_3/',
    # '/Users/yao/Desktop/Data/resolution/Bz60/',
    # '/Users/yao/Desktop/Data/resolution/Bz0/'
    # '/Users/yao/Desktop/Data/dx0.025/apo23_ne5e19_v8200_Bz0_dx0.025_4/',
    # '/Users/yao/Desktop/Data/dx0.025/apo23_ne5e19_v8200_Bz60_dx0.025_4_r2/'
    # '/Users/yao/Desktop/Data/Gabriel_2024/B60_5e19_1um-Copy2/',
    # '/Users/yao/Desktop/F1_test1/F1_test1/',
    # '/Users/yao/Documents/Data/F1_test2_r*/',
    # '/Users/yao/Desktop/LIOR_F1_test3/'，
    '/Users/yao/Desktop/Data/x4/',
    # '/Users/yao/Documents/Data/Gabriel_2024/F1_Al6um_preplasma_r*'
    ]

In [4]:
S0 = happi.Open(wkdir[0], reference_angular_frequency_SI = 2.*np.pi*3e8/0.8e-6)


Loaded simulation '/Users/yao/Desktop/Data/x4/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics


In [5]:
P_rad0 = np.array(S0.RadiationSpectrum(1).getData())


In [6]:
dgamma0 = np.array(S0.RadiationSpectrum(1).getAxis('gamma'))

In [7]:
ene_photon0 = dgamma0*0.511 # [MeV]

In [8]:
Lx0 = S0.namelist.Lsim[0]
Ly0 = S0.namelist.Lsim[1]
Prad_unit = 5.11e5 * 1.6e-19 * 3e10 * 1.1e21 / 0.8 / 6.28 * 1e-4 * 1e-4 # J/s/um [the last 1e-4 is cm->um]


In [9]:
P_rad0_time = np.array(S0.RadiationSpectrum(1,units=['s']).getTimes())
P_rad0_dt = P_rad0_time[1] - P_rad0_time[0] # s

In [10]:
P_rad0_dt

1.3020833333333332e-15

In [11]:
plt.semilogx(ene_photon0,P_rad0[43,:]*Lx0*Ly0*Prad_unit/0.511, '-r', label='w=1.25') # [J/s/um/MeV]
plt.xlabel(r'Photon Energy $\varepsilon_{\gamma}$ (MeV)')
plt.ylabel(r'dP/d$\varepsilon_{\gamma}$ (W/MeV/um)')

Text(0, 0.5, 'dP/d$\\varepsilon_{\\gamma}$ (W/MeV/um)')

2024-09-30 12:34:48.276 Python[66080:912225] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-30 12:34:48.276 Python[66080:912225] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [12]:
# time integration 
U_rad0 = np.sum(P_rad0,axis=0)*Lx0*Ly0*Prad_unit/0.511*P_rad0_dt

In [15]:
# brems = np.loadtxt('/Users/yao/Nextcloud/PROJECTS/Apollon/LIOR_2025/ICS_vs_Brems/Bremmstalung_Energy_spectra.txt',skiprows=1) 
brems = np.loadtxt('/Users/yao/Nextcloud/PROJECTS/Apollon/LIOR_2025/ICS_vs_Brems/Bremmstalung_Energy_spectra2.txt',skiprows=1) # with updated laser (x0.5 intensity)

In [27]:
# plt.figure(figsize=(3.487*2.5, 3.487*2.5*0.618))
plt.figure()
plt.loglog(ene_photon0,U_rad0*3, '-r', label='ICS from SMILEI') # [J/MeV] * 3 um
plt.loglog(brems[:,0],brems[:,1]*3, '--b', label='Brems from model') # [J/MeV] * 3 um
plt.xlabel(r'Photon Energy $\varepsilon_{\gamma}$ (MeV)')
plt.ylabel(r'dU/d$\varepsilon_{\gamma}$ (J/MeV)')
plt.legend()
plt.ylim([1e-5,1])
# plt.ylim([0,0.6])
plt.xlim([0.1,100])
# plt.xlim([0,20])
plt.grid(which='both',linestyle=":",alpha=0.5)
plt.savefig('/Users/yao/Desktop/ICS_Brems_f0.5.png',dpi=600)

In [24]:
brems[:,1]

array([  0.5       ,   0.5995996 ,   0.6991992 ,   0.7987988 ,
         0.8983984 ,   0.997998  ,   1.0975976 ,   1.1971972 ,
         1.2967968 ,   1.3963964 ,   1.495996  ,   1.5955956 ,
         1.69519519,   1.7947948 ,   1.89439439,   1.99399399,
         2.09359359,   2.19319319,   2.29279279,   2.39239239,
         2.49199199,   2.59159159,   2.69119119,   2.79079079,
         2.89039039,   2.98998999,   3.08958959,   3.18918919,
         3.28878879,   3.38838839,   3.48798799,   3.58758759,
         3.68718719,   3.78678679,   3.88638639,   3.98598599,
         4.08558559,   4.18518518,   4.28478479,   4.38438438,
         4.48398398,   4.58358358,   4.68318318,   4.78278278,
         4.88238238,   4.98198198,   5.08158158,   5.18118118,
         5.28078078,   5.38038038,   5.47997998,   5.57957958,
         5.67917918,   5.77877878,   5.87837838,   5.97797798,
         6.07757758,   6.17717718,   6.27677678,   6.37637638,
         6.47597598,   6.57557558,   6.67517517,   6.77

In [16]:
# extract the proton spectrum

S0.Screen(0,units=['MeV'],data_log=True).slide()


#0 - Number density of species # 3
    ekin from 0.195695 to 978.474 in 2000 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0] * grid_length[1]



/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [58]:
## prepare the data for the electron energy spectrum

data_e = np.array(S0.ParticleBinning(7,units=['MeV','fs'],
                                #    data_log=True,
                #    xmin=0,xmax=100
                   ).getData())

energy_e = np.array(S0.ParticleBinning(7,units=['MeV','fs'],data_log=True,
                   xmin=0,xmax=100).getAxis("ekin"))

# the time to choose
Time_e = np.array(S0.ParticleBinning(7,units=['MeV','fs'],data_log=True,
                   xmin=0,xmax=100).getTimes())

In [59]:
# make sure the time chosen is "OK"
ii=50
Time[ii]

65.10416666666666

In [61]:
# check what it looks like, with fancy units
plt.semilogy(energy_e,data_e[ii,:]*1.1e21/(0.8*0.8)/(5.11e5*1.6e-19/1.6e-19/1e6))
plt.xlabel('Energy (MeV)')
plt.ylabel('dN/dE (cm^-3/MeV)')
plt.xlim([0,150])
plt.ylim([1e16,1e20])

(1e+16, 1e+20)

In [63]:
np.savetxt('/Users/yao/Desktop/data_e.txt',data_e[ii,:]) # save the data but w/o the units above
np.savetxt('/Users/yao/Desktop/energy_e.txt',energy_e)

In [18]:
energy = np.array(S0.ParticleBinning(3,units=['MeV','fs'],data_log=True,
                   xmin=0,xmax=100).getAxis("ekin"))

In [19]:
Time = np.array(S0.ParticleBinning(3,units=['MeV','fs'],data_log=True,
                   xmin=0,xmax=100).getTimes())

In [56]:
ii=50
Time[ii]

65.10416666666666

In [47]:
plt.semilogy(energy,data[ii,:]*1.1e21/(0.8*0.8)/(5.11e5*1.6e-19/1.6e-19/1e6))
plt.xlabel('Energy (MeV)')
plt.ylabel('dN/dE (cm^-3/MeV)')
plt.xlim([0,51])
plt.ylim([1e16,1e20])

(1e+16, 1e+20)

In [49]:
# unit for dN/dE from SMILEI:
# N_r / K_r
# N_r = 1.1e21 / lmd[um]^2 [cm^-3]
# K_r = m_e c^2 = 5.11e5 * 1.6e-19 [J] 

# from cm^-3 / J to Number of Particle / MeV
# N_A = 6.022e23 mol^-1
# J -> MeV --> J/qe/1e6

# don't forget the Lx*Ly factor from the ParticleBinning

energy_spectrum_unit = 1.1e21/(0.8*0.8) / (5.11e5*1.6e-19/1.6e-19/1e6) * Lx0 * Ly0 

In [50]:
np.savetxt('/Users/yao/Desktop/data.txt',data[113,:]*1.1e21/(0.8*0.8)/(5.11e5*1.6e-19/1.6e-19/1e6))

In [51]:
np.savetxt('/Users/yao/Desktop/energy.txt',energy)

In [54]:
S0.ParticleBinning(3,units=['MeV','fs','cm^-3/MeV'],
                                   data_log=True,
                   xmin=0,xmax=52
                   ).slide()


#3 - Number density of species # 3
    ekin from 0.000195695 to 978.474 in 2000 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0] * grid_length[1]



/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [ ]:
S0.ParticleBinning(5, )